# Projeto 1 - Ciência dos Dados

Nome: Arthur Gomes Chieppe

Nome: João Pedro Di Jardim Marquez

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from emoji import UNICODE_EMOJI

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'PS5.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.tail(5)

,Treinamento,Relevância:,Treinamento Limpo
295,discord https://t.co/q6oxummoxf\nsiga nossas m...,0,discord https//t.co/q6oxummoxfsiga nossas midi...
296,se tivesse a ps5 despedia me da marta e dos me...,0,se tivesse a ps5 despedia me da marta e dos me...
297,"desculpa pai, mas ps5 vai ter que vir no final...",0,desculpa pai mas ps5 vai ter que vir no final ...
298,"eu nem tô assim, quem tá são vocês\n\n#ps5show...",0,eu nem tô assim quem tá são vocês#ps5showcase ...
299,as vezes dá medo de pensar que o #ps5 chegue n...,1,as vezes dá medo de pensar que o #ps5 chegue n...


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)


,Teste,Relevância:
0,"sony nega, mas ações continuam em queda após r...",1.0
1,hoje tem \n\n#ps5,0.0
2,@prodigyrr não faça isso. ps5 ta uma completa ...,0.0
3,"hoje lançam o preço do ps5, tô só por saber qu...",1.0
4,"@1playergirl bom, nesse momento me sinto num h...",0.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto trata-se do console de videogames PlayStation5. Quando extraímos os Tweets, havia uma espera coletiva acerca da livestream de lançamento do novo PlayStation 5. Assim , consideramos relevante especulações acerca do preço do novo console, bem como opiniões dos usuários sobre seu design. Ademais, consideramos algumas comparações entre o novo PlayStation e Xbox como relevantes, porém somente aquelas bem fundamentadas e intencionadas, já que consideramos irrelevantes comparações vagas e com discurso de ódio. Anúncios feitos acerca da live também foram consideradas como relevantes, com exceção de anúncios que tinham como objetivo somente a autopromoção dos usuários.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
#Limpando caracteres:
def limpa_tweets(dataframe):
    train_limpo = dataframe.iloc[:, 0]
    ls= []
    for w in train_limpo:
        w = w.replace(":",'')
        w = w.replace('"','')
        w = w.replace("'",'')
        w = w.replace("(",'')
        w = w.replace(")",'')
        w = w.replace('!','')
        w = w.replace('...','')
        w = w.replace(',','')
        w = w.replace('\n','')
        w = ''.join(' ' + char+' ' if char in UNICODE_EMOJI else char for char in w)
        w = w.replace('  ',' ')
        w = w.strip()
        w = w.casefold()
        ls.append(w)
    dataframe['Treinamento Limpo'] = ls
    return dataframe
train = limpa_tweets(train)


In [6]:
#train.to_excel(r'PS5Emoji.xlsx', sheet_name='Treinamento', index = False)

## Classificador:
Precisamos de:
P(R|T) = P(T|R) * P(R)/ P(T)




In [13]:
#Calculando Probabilidades:
def treinamento(dataframe):
    Rele = pd.Series(dtype=object)
    Irrele = pd.Series(dtype=object)

    for s in dataframe['Treinamento Limpo']:
        lsWords = pd.Series(s.split())
        for i in range(len(lsWords)):
            if dataframe['Relevância:'][i+1] == 1:
                Rele = Rele.append(lsWords)
            else:
                Irrele = Irrele.append(lsWords)
    return [Rele, Irrele]
    



In [17]:
Rele = treinamento(train)[0]
Irrele = treinamento(train)[1]
total = Irrele.count()+Rele.count()

#Calcaulando probabilidade em tweets relevantes:
lsRele = []
for v in Rele.value_counts():
    lsRele.append((v+1)/(total+Rele.count()))
lsKeys = Rele.value_counts().index.tolist()
dic_rele = {lsKeys[i]: lsRele[i] for i in range(len(lsKeys))} 
print(dic_rele)







{'o': 0.011339096252433383, 'e': 0.009132620166878446, 'ps5': 0.008268808252533535, 'de': 0.008040336260695932, 'que': 0.007840032048673923, 'do': 0.007758658462539983, 'a': 0.007408126091501468, 'um': 0.004998216040611679, 'é': 0.004488066250618127, 'no': 0.004328448831663089, 'dia': 0.0042408157389034595, 'na': 0.003251813692044793, 'bom': 0.003148531832720945, 'com': 0.0029983036737044385, 'vai': 0.0027604424219283036, 'da': 0.0027385341487383965, 'hoje': 0.002694717602358582, 'não': 0.002641511796040236, 'pra': 0.002560138209906295, 'eu': 0.002513191910213637, 'mais': 0.0024192993108283203, 'evento': 0.0021157132394824638, 'sony': 0.002090675212979713, 'uma': 0.0020468586665998985, 'preço': 0.0018215164280751392, '#ps5': 0.001774570128382481, 'para': 0.0016337312293045062, 'tem': 0.0016306014759916623, 'me': 0.0015711361630476287, 'em': 0.001561746903109097, 'se': 0.0014615947970980927, 'q': 0.0013833509642769957, 'xbox': 0.001292588118204523, 'os': 0.0012894583648916792, 'por': 0.

In [18]:
#Calculando Probabilidade em tweets irrelevantes:
lsIrrele = []
for v in Irrele.value_counts():
    lsIrrele.append((v+1)/(total+Irrele.count()))
lsKeys = Irrele.value_counts().index.tolist()
dic_irrele = {lsKeys[i]: lsIrrele[i] for i in range(len(lsKeys))} 
print(dic_irrele)

{'o': 0.011063609439431158, 'e': 0.009216517110074074, 'de': 0.008054585525110984, 'que': 0.007675694790883889, 'do': 0.0074894068465555675, 'ps5': 0.007394684162998794, 'a': 0.007331535707294278, 'um': 0.004976098309515841, 'é': 0.004366715711967264, 'dia': 0.004341456329685457, 'no': 0.0043098821018332, 'na': 0.0033216087700575284, 'bom': 0.0032679325827086898, 'com': 0.0030153387598906268, 'da': 0.0027216984408646285, 'vai': 0.002677494521871468, 'hoje': 0.002566984724388565, 'não': 0.0025480401876772103, 'pra': 0.002475419463617017, 'eu': 0.002453317504120437, 'mais': 0.0023680670889193404, 'sony': 0.0020333802736854072, 'uma': 0.0020333802736854072, 'evento': 0.002017593159759278, 'preço': 0.0017239528407332798, '#ps5': 0.0016671192305992156, 'para': 0.0016260727343912805, 'me': 0.0015944985065390225, 'em': 0.0015534520103310874, 'tem': 0.0015502945875458616, 'se': 0.00142399767613683, 'q': 0.0013450621065061853, 'os': 0.0013040156102982503, 'por': 0.0012850710735868955, 'isso': 0

In [57]:

dataframe = train
for t in dataframe:
    lsWords = t.split()
    for w in lsWords:
    

TypeError: 'method' object is not subscriptable

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**